In [ ]:
df = pd.read_csv('data/311_Fireworks_Complaints.csv',parse_dates=['Created Date', 'Closed Date'],
     infer_datetime_format=True)
df.head(5)

# Data cleanup

In [ ]:
df.isnull().sum()

In [ ]:
delete_columns = ["Intersection Street 1", "Intersection Street 2", "Address Type", "Landmark", "Facility Type",\
                  "Status", "X Coordinate (State Plane)", "Y Coordinate (State Plane)", "Park Facility Name",\
                  "Park Borough", "School Name", "School Number", "School Region", "School Code",\
                  "School Phone Number", 
                  "School Address", "School City", "School State", "School Zip", "School Not Found",\
                  "School or Citywide Complaint", "Vehicle Type", "Taxi Company Borough", "Taxi Pick Up Location",\
                  "Bridge Highway Name", "Bridge Highway Direction", "Road Ramp", "Bridge Highway Segment",\
                  "Garage Lot Name", "Ferry Direction", "Ferry Terminal Name","Due Date", \
                  "Resolution Action Updated Date","Incident Address","Street Name","Cross Street 1","Cross Street 2",\
                  "Location Type"]

df.drop(delete_columns, axis=1, inplace=True)
df.shape

In [ ]:
#Drop NAs in several categories and print new null sums
df = df.dropna(subset=['Latitude',"Longitude","Complaint Type", "Incident Zip"], how='any')
df.isnull().sum()

In [ ]:
#Change incident_zip column datatype from float to str
#to map with GeoJSON
df['Incident Zip'] = df['Incident Zip'].astype(int)
df['Incident Zip'] = df['Incident Zip'].astype(str)
df['Incident Zip'].dtype

In [ ]:
#Cleaning column name syntax
df.columns = ['unique_key','created_date', 'closed_date', 'agency', 'agency_name','complaint_type',\
                    'descriptor', 'incident_zip', 'city','community_board', 'borough', 'latitude', 'longitude',\
                    'location']
df.head(5)

In [ ]:
df.to_csv('cleaned_fireworks.csv')

# Data analysis

In [2]:
#count by borough
agency_count = sns.countplot(x="borough", data=df)

NameError: name 'df' is not defined

In [ ]:
# List of months by number.
# timestamps = [timestring.parse(row[2]) for row in df.itertuples()]
df['month'] = df['created_date'].apply(lambda x: x.month)
months = [row.month for row in df.itertuples()]
#print(sorted(months))

# Define the plot
plot = Histogram(months, xlabel='Months', ylabel='Frequency', title='Firework Complaints by Month')
handle = show(plot, notebook_handle=True)  # Show the figure.

In [ ]:
df['hour'] = df['created_date'].apply(lambda x: x.hour)

hour_counts = df.groupby(['hour'],sort = True).count()
# Plot complaint counts
hour_counts.plot(kind = 'area', stacked = False,
                  color = 'darkorange', linewidth = 2, legend = False)
plt.tick_params(axis = 'both', which = 'major', labelsize = 18)
plt.title('Number of complaints / hour of the day\n')
plt.xlabel('')
plt.ylabel('Number of complaints')

# Choropleth map

In [ ]:
#choropleth map
zipcode_geo = r'data/ZIPNYC.geojson'

zip_data = df["incident_zip"].value_counts()
zip_count = pd.DataFrame(zip_data).reset_index()
zip_count.columns = ["postalCode","count"]


#Let Folium determine the scale
map = folium.Map(location=[40.7128, -74.0059], zoom_start=10)
map.choropleth(geo_path=zipcode_geo, data=zip_count,
             columns=["postalCode","count"],
             key_on='feature.properties.postalCode',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2)

map.save('html/fireworks_by_zipcode.html')
map

# Cluster map

In [ ]:
from folium import plugins

marker_cluster = folium.MarkerCluster().add_to(map)
for name, row in df.iterrows():
    folium.Marker([row["latitude"], row["longitude"]], popup="{0}".format(row["incident_zip"])).add_to(marker_cluster)
    
map.save('html/firework_cluster_map.html')
map